# [602. Friend Requests II: Who Has the Most Friends](https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/?envType=study-plan-v2&envId=top-sql-50)

Table: RequestAccepted

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |
+----------------+---------+</pre>
(requester_id, accepter_id) is the primary key (combination of columns with unique values) for this table.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date when the request was accepted.


Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

The result format is in the following example.



Example 1:

Input:
RequestAccepted table:
<pre>+--------------+-------------+-------------+
| requester_id | accepter_id | accept_date |
+--------------+-------------+-------------+
| 1            | 2           | 2016/06/03  |
| 1            | 3           | 2016/06/08  |
| 2            | 3           | 2016/06/08  |
| 3            | 4           | 2016/06/09  |
+--------------+-------------+-------------+</pre>
Output:
<pre>+----+-----+
| id | num |
+----+-----+
| 3  | 3   |
+----+-----+</pre>
Explanation:
The person with id 3 is a friend of people 1, 2, and 4, so he has three friends in total, which is the most number than any others.


Follow up: In the real world, multiple people could have the same most number of friends. Could you find all these people in this case?

In [8]:
# pandas schema

import pandas as pd

data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype(
    {'requester_id': 'Int64', 'accepter_id': 'Int64', 'accept_date': 'datetime64[ns]'})

#to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

request_accepted_df = spark.createDataFrame(request_accepted)
request_accepted_df.show()

+------------+-----------+-------------------+
|requester_id|accepter_id|        accept_date|
+------------+-----------+-------------------+
|           1|          2|2016-06-03 00:00:00|
|           1|          3|2016-06-08 00:00:00|
|           2|          3|2016-06-08 00:00:00|
|           3|          4|2016-06-09 00:00:00|
+------------+-----------+-------------------+



In [27]:
request_accepted_df.select(col('requester_id').alias('id'))\
        .union(request_accepted_df.select('accepter_id'))\
        .groupBy('id').agg(count('id').alias('num'))\
        .orderBy(col('num').desc())\
        .limit(1)\
        .show()

+---+---+
| id|num|
+---+---+
|  3|  3|
+---+---+



In [7]:
# Solving in Spark SQL

request_accepted_df.createOrReplaceTempView('RequestAccepted')

spark.sql('''
select id,count(id) as num
from(
    select requester_id as id from RequestAccepted
    union all
    select accepter_id from RequestAccepted
    )x
group by id
order by num desc
limit 1
''').show()

+---+---+
| id|num|
+---+---+
|  3|  3|
+---+---+



In [ ]:
spark.stop()